# Semantic Kernel 

In this code sample, you will use the [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework to create a basic agent. 

The goal of this sample is to show you the steps that we will later use in the addtional code samples when implementing the different agentic patterns. 

## Import the Needed Python Packages 

In [1]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv



from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory


from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

from semantic_kernel.connectors.ai import FunctionChoiceBehavior

from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function

## Creating the Client and Kernel 

In this sample, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for access to the LLM. 

The `ai_model_id` is defined as `gpt-4o-mini`. Try changing the model to another model available on the GitHub Models marketplace to see the different results. 

For us to us the `Azure Inference SDK` that is used for the `base_url` for GitHub Models, we will use the `AsyncOpenAI` connector within Semantic Kernel. There are also other [available connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) to use Semantic Kernel for other model providers.

We will also create a `Kernel`. A `kernel` is a collection of the services and plugins that will be used by your Agents. In this snipppet, we are creating the kernel and adding the `chat_completion_service` to it.  

In [4]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [5]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), base_url="https://models.inference.ai.azure.com/")

kernel = Kernel()
kernel.add_plugin(DestinationsPlugin(), plugin_name="destinations")

service_id = "agent"

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
    service_id=service_id
)
kernel.add_service(chat_completion_service)

## Creating the Agent 

Below we will are creating the Agent called `TravelAgent` and also creating a variable called `AGENT_INSTRUCTIONS`. We will later add this to our `system_message` that will give the agent instructions on the task, behavior and tone.

For this example, we are using very simple instructions. You can change these instructions to see how the agent responds differently. 

In [6]:
settings = kernel.get_prompt_execution_settings_from_service_id(
    service_id=service_id)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

In [7]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = "You are a helpful AI Agent that can help plan vacations for customers at random destinations"
agent = ChatCompletionAgent(
    service_id=service_id, 
    kernel=kernel, 
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings=settings)
)

## Running the Agents 

Now we can run the Agent by defining the `ChatHistory` and adding the `system_message` to it. We will use the `AGENT_INSTRUCTIONS` that we defined earlier. 

After these are defined, we create a `user_inputs` that will be what the user is sending to the agent. In this case, we have set this message to `Plan me a sunny vacation`. 

Feel free to change this message to see how the agent responds differently. 

In [8]:
from IPython.display import display, HTML


async def main():
    # Define the chat history
    chat_history = ChatHistory()

    # Respond to user input
    user_inputs = [
        "Plan me a day trip.",
        "I don't like that destination. Plan me another vacation.",
    ]

    for user_input in user_inputs:
        # Add the user input to the chat history
        chat_history.add_user_message(user_input)

        # Start building HTML output
        html_output = f"<div style='margin-bottom:10px'>"
        html_output += f"<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += f"</div>"

        agent_name: str | None = None
        full_response = ""
        function_calls = []
        function_results = {}

        # Collect the agent's response with function call tracking
        async for content in agent.invoke_stream(chat_history):
            if not agent_name and hasattr(content, 'name'):
                agent_name = content.name

            # Track function calls and results
            for item in content.items:
                if isinstance(item, FunctionCallContent):
                    call_info = f"Calling: {item.function_name}({item.arguments})"
                    function_calls.append(call_info)
                elif isinstance(item, FunctionResultContent):
                    result_info = f"Result: {item.result}"
                    function_calls.append(result_info)
                    # Store function results
                    function_results[item.function_name] = item.result

            # Add content to response if it's not a function-related message
            if (hasattr(content, 'content') and content.content and content.content.strip() and
                not any(isinstance(item, (FunctionCallContent, FunctionResultContent))
                        for item in content.items)):
                full_response += content.content

        # Add function calls to HTML if any occurred
        if function_calls:
            html_output += f"<div style='margin-bottom:10px'>"
            html_output += f"<details>"
            html_output += f"<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
            html_output += f"<div style='margin:10px; padding:10px; background-color:#f8f8f8; border:1px solid #ddd; border-radius:4px; white-space:pre-wrap;'>"
            html_output += "<br>".join(function_calls)
            html_output += f"</div></details></div>"

        # Add agent response to HTML
        html_output += f"<div style='margin-bottom:20px'>"
        html_output += f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
        html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{full_response}</div>"
        html_output += f"</div>"
        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()